In [8]:
!ls ../input

x-small-balanced-csv


## Подготовим датасет

In [ ]:
!unzip ./arxivData.json.zip

Archive:  ./arxivData.json.zip
  inflating: arxivData.json          


In [ ]:
import pandas as pd
Xjs = pd.read_json('./arxivData.json')
Xjs.head()

,author,day,id,link,month,summary,tag,title,year
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech S...",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We propose an architecture for VQA which utili...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Dual Recurrent Attention Units for Visual Ques...,2018
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck De...",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Recent approaches based on artificial neural n...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Sequential Short-Text Classification with Recu...,2016
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim...",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,We introduce the multiresolution recurrent neu...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Multiresolution Recurrent Neural Networks: An ...,2016
3,"[{'name': 'Sebastian Ruder'}, {'name': 'Joachi...",23,1705.08142v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Multi-task learning is motivated by the observ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning what to share between loosely related...,2017
4,"[{'name': 'Iulian V. Serban'}, {'name': 'Chinn...",7,1709.02349v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We present MILABOT: a deep reinforcement learn...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Reinforcement Learning Chatbot,2017


In [8]:
import string
import ast

def clean_s(s: str) -> str:
  translator = str.maketrans('','',string.punctuation)
  return s.lower().translate(translator).replace('\n', '')

def authors_get(s: str) -> str:
  cells = ast.literal_eval(s)
  return ','.join([x['name'].lower() for x in cells])

def tags_get(s: str) -> str:
  cells = ast.literal_eval(s)
  return ','.join([x['term'] for x in cells])

In [ ]:
print(clean_s(Xjs['summary'][0]))
print(authors_get(Xjs['author'][0]))
print(Xjs['tag'][0])
print(tags_get(Xjs['tag'][0]))

Xjs['x'] =                              \
  Xjs['author'].map(authors_get)         \
  + "|"                                \
  + Xjs['title'].map(clean_s)           \
  + "|"                                  \
  + Xjs['summary'].map(clean_s)


Xjs['labels_s'] = Xjs['tag'].map(tags_get)

print(f"example of row:\n{Xjs['x'][0]}")

NameError: name 'Xjs' is not defined

In [ ]:
Xful = Xjs[['x', 'labels_s']]
Xful.head()
Xful.to_csv('Xful.csv', index=False)

In [ ]:
Xsmal = Xjs[:2000][['x', 'labels_s']]
Xsmal.to_csv('Xsmal.csv', index=False)

## Модель

In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
# Xful = pd.read_csv('./Xful.csv')
# Xful['labels_s'] =  Xful['labels_s'].map(lambda r: r.split(',')[0])
# print(len(Xful))
# sampled_df = Xful.groupby('labels_s').sample(n=50, replace=True, random_state=42)
# print(sampled_df)

# sampled_df.to_csv('X-small-balanced.csv')

41000
                                                       x  labels_s
34792  jens c. astor,christoph adami|development and ...  adap-org
34793  charles ofria,christoph adami|evolution of gen...  adap-org
34792  jens c. astor,christoph adami|development and ...  adap-org
34792  jens c. astor,christoph adami|development and ...  adap-org
34792  jens c. astor,christoph adami|development and ...  adap-org
...                                                  ...       ...
37317  claudio mazzola|generalised reichenbachian com...   stat.OT
37916  mihai cucuringu,jesus puente,david shue|model ...   stat.OT
35899  joris m. mooij,dominik janzing,bernhard schölk...   stat.OT
35899  joris m. mooij,dominik janzing,bernhard schölk...   stat.OT
37534  richard pettigrew|aggregating incoherent agent...   stat.OT

[6300 rows x 2 columns]


In [16]:
!ls ../input/x-small-balanced-csv

X-small-balanced.csv


In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

X = pd.read_csv('../input/x-small-balanced-csv/X-small-balanced.csv')
X.sample(10)

,Unnamed: 0,x,labels_s
610,37795,"dmitri b. chklovskii,alexei a. koulakov|ocular...",cond-mat.soft
386,33912,andrew kehler|common topics and coherent situa...,cmp-lg
325,39241,"kevin r. moon,veronique delouille,jimmy j. li,...",astro-ph.SR
4770,38801,"stephane roux,hugo leclerc,françois hild|effic...",physics.class-ph
1999,37710,"babatunde opeoluwa akinkunmi,moyin florence ba...",cs.MA
1402,37381,"jon kleinberg,sendhil mullainathan,johan ugand...",cs.DS
5477,34943,"lars marstaller,arend hintze,christoph adami|t...",q-bio.NC
1400,39266,ze wang|mrfzoom a fast dictionary searching al...,cs.DS
1500,8928,anssi yli-jyrä|the power of constraint grammar...,cs.FL
2669,38175,"jan hauffa,tobias lichtenberg,georg groh|an ev...",cs.SI


In [2]:
lens = []
for r in X['x'].values:
  lens.append(len(re.split(', | ', r)))
print(f"len L (in tokens):")
print(f"{np.min(lens)} < L~{np.mean(lens)} < {np.max(lens)}")

X['labels_s'] = X['labels_s'].map(lambda r: r.split(',')[0])
classes = set(X['labels_s'].values)
c2v = {c:v for v,c in enumerate(classes)}
v2c = {v:c for v,c in enumerate(classes)}

print(f"n-classes:{len(classes)}")
X['labels'] = X['labels_s'].map(lambda r: c2v[r])

X.sample(10)

len L (in tokens):
11 < L~146.0806349206349 < 429
n-classes:126


,Unnamed: 0,x,labels_s,labels
315,39748,"a. asensio ramos,i. s. requerey,n. vitas|deepv...",astro-ph.SR,84
573,34802,"s. d. makovetskiy,d. n. makovetskii|a computat...",cond-mat.other,88
2076,12229,"matthias seeger,asmus hetzel,zhenwen dai,neil ...",cs.MS,109
2717,11047,"dorsa sadigh,henrik ohlsson,s. shankar sastry,...",cs.SY,25
130,12278,"siamak ravanbakhsh,junier oliva,sebastien from...",astro-ph.CO,11
5719,37267,"lamb wubin,naixin ren|decision structure of ri...",q-fin.EC,87
584,34802,"s. d. makovetskiy,d. n. makovetskii|a computat...",cond-mat.other,88
5270,7971,"zhen li,sheng wang,yizhou yu,jinbo xu|predicti...",q-bio.BM,112
3767,34822,"alexei d. miasnikov,alexei g. myasnikov|whiteh...",math.GR,53
3811,10637,jesse johnson|topological graph clustering wit...,math.GT,89


In [4]:
!pip install transformers==4.50.3
!pip show transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 68.4 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
Name: transformers
Version: 4.50.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: kaggle-environments, peft, sentence-transformers


In [5]:
# from transformers import RobertaTokenizer, RobertaForSequenceClassification
import transformers as tr
print(tr.__version__)
t = tr.RobertaTokenizer.from_pretrained("roberta-base")
m = tr.RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(classes)).to('cuda')

4.50.3


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
text = clean_s("hello.")
inputs = t(text, return_tensors="pt")
p_c = m(**inputs.to('cuda')).logits.argmax(-1)
print(f"p_c:{p_c}")
print(f"o:{v2c[int(p_c)]}")

p_c:tensor([107], device='cuda:0')
o:physics.hist-ph


In [10]:
import torch.utils.data as d
from datasets import Dataset, DatasetDict
import torch

ds = Dataset.from_dict({
  'x': X['x'].values,
  'labels_s': X['labels_s'].values,
  'labels': X['labels'].values
}).train_test_split(test_size=.2)

class SeqDs(d.Dataset):
  def __init__(self, ds):
    self.ds = ds

  def __len__(self):
    return len(self.ds['input_ids'])

  def __getitem__(self, i):
    return {
      "input_ids":
        torch.tensor(self.ds["input_ids"][i]),
      "attention_mask":
        torch.tensor(self.ds["attention_mask"][i]),
      "labels":
        torch.tensor(self.ds["labels"][i])
    }

def preprocess(rs):
  t_o = t(
      rs['x'],
      max_length=315,
      padding='max_length',
      truncation=True,
      return_tensors='pt',
  )
  return {
      "input_ids": t_o.input_ids,
      "attention_mask": t_o.attention_mask,
      "labels": torch.tensor(rs['labels'])
    }

tr_ds = SeqDs(preprocess(ds['train']))
ev_ds = SeqDs(preprocess(ds['test']))
# tr_ds[0]

In [11]:
# tr_ds[300]

In [12]:
# !pip install --upgrade transformers
# !pip uninstall transformers -y
# !pip install transformers==4.50.3

In [13]:
# import sys
# sys.modules.pop('transformers', None)

# import transformers as t
print(tr.__version__)
# from transformers import Trainer, TrainingArguments
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
# !wandb -login --relogin
# import wandb
# import torch.nn as nn
# wandb.init(project="roberta-classification", name='view')

4.50.3


In [43]:
import wandb
wandb.login(key='3d142fb56b227470f028e325e8c69e48dfae7ff1')
import torch.nn as nn
wandb.init(project="roberta-classification-kaggle", name='view')

args = tr.TrainingArguments(
  output_dir="./roberta-classification",
  evaluation_strategy="steps",
  save_strategy="steps",
  num_train_epochs=30,
  learning_rate=1e-2,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  weight_decay=0.01,

  eval_steps=50,
  save_total_limit=2,

  logging_dir="./logs",
  # load_best_model_at_end=True,
  report_to='wandb',
)

def compute_metrics(pred):
  # print(pred)
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc}

trainer = tr.Trainer(
  model=m,
  args=args,
  train_dataset=tr_ds,
  eval_dataset=ev_ds,
  compute_metrics=compute_metrics,
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [44]:
# from transformers import TrainerState

# Example usage that might cause the error
print(tr.__version__)
# trainer_state = tr.TrainerState(best_global_step=0)

4.50.3


In [45]:
# !ls ../input/m/pytorch/default/1/roberta-classification
!ls ./roberta-classification
# !ls ../input/m/pytorch/default/1/roberta-classification
# pt = "../input/m/pytorch/default/1/roberta-classification/checkpoint-3150"
pt= './roberta-classification/checkpoint-7875'

checkpoint-7500  checkpoint-7875


In [46]:
# !pip install transformers==4.50.3
# !pip show transformers
# !pip show torchvision
# !pip show transformers

In [47]:
trainer.train(resume_from_checkpoint=pt)

Step,Training Loss,Validation Loss,Accuracy
7900,4.235300,4.264178,0.091270
7950,4.235300,4.270469,0.082540
8000,4.203600,4.303651,0.065873
8050,4.203600,4.271545,0.076984
8100,4.203600,4.241523,0.092063
8150,4.203600,4.259571,0.068254
8200,4.203600,4.241198,0.080952
8250,4.203600,4.251227,0.092857
8300,4.203600,4.241065,0.100794
8350,4.203600,4.266409,0.077778


TrainOutput(global_step=9450, training_loss=0.6936814662388393, metrics={'train_runtime': 1318.3482, 'train_samples_per_second': 114.689, 'train_steps_per_second': 7.168, 'total_flos': 2.45286733374144e+16, 'train_loss': 0.6936814662388393, 'epoch': 30.0})

In [51]:
n=150
k=50
sample = [x for x in X['x'][n:n+k]]
print(f"true labels: {[x for x in X['labels'][n:n+k]]}")

test_ins = [t(x, return_tensors="pt") for x in sample]
p_c = [int(m(**ii.to('cuda')).logits.argmax(-1)) for ii in test_ins]
# print(m(**test_ins[0].to('cuda')).logits)
print(f"{p_c}")


true labels: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [ ]:
from collections import Counter

cnt = Counter
print(cnt(X['labels_s'].values))
print(f"tag of popular:{v2c[24]}")

Counter({'adap-org': 50, 'astro-ph': 50, 'astro-ph.CO': 50, 'astro-ph.EP': 50, 'astro-ph.GA': 50, 'astro-ph.IM': 50, 'astro-ph.SR': 50, 'cmp-lg': 50, 'cond-mat': 50, 'cond-mat.dis-nn': 50, 'cond-mat.mtrl-sci': 50, 'cond-mat.other': 50, 'cond-mat.soft': 50, 'cond-mat.stat-mech': 50, 'cond-mat.supr-con': 50, 'cs.AI': 50, 'cs.AR': 50, 'cs.CC': 50, 'cs.CE': 50, 'cs.CG': 50, 'cs.CL': 50, 'cs.CR': 50, 'cs.CV': 50, 'cs.CY': 50, 'cs.DB': 50, 'cs.DC': 50, 'cs.DL': 50, 'cs.DM': 50, 'cs.DS': 50, 'cs.ET': 50, 'cs.FL': 50, 'cs.GL': 50, 'cs.GR': 50, 'cs.GT': 50, 'cs.HC': 50, 'cs.IR': 50, 'cs.IT': 50, 'cs.LG': 50, 'cs.LO': 50, 'cs.MA': 50, 'cs.MM': 50, 'cs.MS': 50, 'cs.NA': 50, 'cs.NE': 50, 'cs.NI': 50, 'cs.OH': 50, 'cs.OS': 50, 'cs.PF': 50, 'cs.PL': 50, 'cs.RO': 50, 'cs.SC': 50, 'cs.SD': 50, 'cs.SE': 50, 'cs.SI': 50, 'cs.SY': 50, 'econ.EM': 50, 'eess.AS': 50, 'eess.IV': 50, 'eess.SP': 50, 'gr-qc': 50, 'hep-ex': 50, 'hep-lat': 50, 'hep-ph': 50, 'hep-th': 50, 'math.AG': 50, 'math.AP': 50, 'math.AT': 5

In [56]:
!ls ./roberta-classification/checkpoint-9450

config.json	   optimizer.pt   scheduler.pt	      training_args.bin
model.safetensors  rng_state.pth  trainer_state.json


In [62]:
# !rm -R ./results/
!zip -r roberta-9450.zip ./roberta-classification/checkpoint-9450

  adding: roberta-classification/checkpoint-9450/ (stored 0%)
  adding: roberta-classification/checkpoint-9450/config.json (deflated 75%)
  adding: roberta-classification/checkpoint-9450/training_args.bin (deflated 52%)
  adding: roberta-classification/checkpoint-9450/trainer_state.json (deflated 86%)
  adding: roberta-classification/checkpoint-9450/optimizer.pt (deflated 26%)
  adding: roberta-classification/checkpoint-9450/rng_state.pth (deflated 25%)
  adding: roberta-classification/checkpoint-9450/model.safetensors (deflated 12%)
  adding: roberta-classification/checkpoint-9450/scheduler.pt (deflated 56%)


In [65]:
# !cp ./roberta-9450.zip ../input
!ls

roberta-9450.zip  roberta-classification  wandb
